In [1]:
# imports 
import numpy as np
from math import sqrt 
from pprint import pprint
from numpy import array
from scipy.io import arff
import pandas as pd

In [2]:
# load data

df=pd.read_csv('Wilt_withoutdupl_norm_05.csv')  

del df['id']
del df['Unnamed: 0']
df['outlier'] = df.outlier.apply(lambda label: 1 if label == "'yes'" else 0)

df.head()

,GLCM_pan,Mean_Green,Mean_Red,Mean_NIR,SD_pan,outlier
0,0.656711,0.050984,0.044570,0.218476,0.132110,1
1,0.680591,0.049425,0.041939,0.177275,0.106749,1
2,0.734892,0.047396,0.042926,0.259034,0.143741,1
3,0.698087,0.035317,0.027066,0.127065,0.095697,1
4,0.738927,0.046076,0.040228,0.295501,0.112481,1


In [3]:
df.shape

(4819, 6)

In [4]:
# data to values
df = df.values

In [5]:
# Extratrees Deep
import argparse
import sys
import pickle
from sklearn.metrics import accuracy_score
sys.path.insert(0, "lib")
from gcforest.gcforest import GCForest
from gcforest.utils.config_utils import load_json
config = load_json("./examples/Wilt.json")
gc = GCForest(config)

In [7]:
# train test 
from sklearn.cross_validation import train_test_split
y = df[:,5]
X = df[:,0:5]

X_train,X_test,y_train,y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [7]:
# number of class
len(np.unique(y_train))

2

In [8]:
             # X_enc is the concatenated predict_proba result of ExtraTrees

X_train_enc, X_test_enc = gc.fit_transform(X_train, y_train, X_test=X_test, y_test=y_test)
   

[ 2018-04-25 01:05:26,613][cascade_classifier.fit_transform] X_groups_train.shape=[(3228, 5)],y_train.shape=(3228,),X_groups_test.shape=[(1591, 5)],y_test.shape=(1591,)
[ 2018-04-25 01:05:26,614][cascade_classifier.fit_transform] group_dims=[5]
[ 2018-04-25 01:05:26,616][cascade_classifier.fit_transform] group_starts=[0]
[ 2018-04-25 01:05:26,618][cascade_classifier.fit_transform] group_ends=[5]
[ 2018-04-25 01:05:26,623][cascade_classifier.fit_transform] X_train.shape=(3228, 5),X_test.shape=(1591, 5)
[ 2018-04-25 01:05:26,625][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(3228, 5), X_cur_test.shape=(1591, 5)
[ 2018-04-25 01:05:27,263][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=97.52%
[ 2018-04-25 01:05:28,046][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=98.45%
[ 2018-04-25 01:05:28,820][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_fold

In [9]:
# predict
y_pred = gc.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print("Test Accuracy of ExtraTrees = {:.6f} %".format(acc * 100))

[ 2018-04-25 01:05:34,570][cascade_classifier.transform] X_groups_test.shape=[(1591, 5)]
[ 2018-04-25 01:05:34,572][cascade_classifier.transform] group_dims=[5]
[ 2018-04-25 01:05:34,574][cascade_classifier.transform] X_test.shape=(1591, 5)
[ 2018-04-25 01:05:34,576][cascade_classifier.transform] [layer=0] look_indexs=[0], X_cur_test.shape=(1591, 5)


Test Accuracy of ExtraTrees = 98.114393 %


In [10]:
# Make data + estimators
X_test_enc = gc.transform(X_test)
X_train_enc = X_train_enc.reshape((X_train_enc.shape[0], -1))
X_test_enc = X_test_enc.reshape((X_test_enc.shape[0], -1))
X_train_origin = X_train.reshape((X_train.shape[0], -1))
X_test_origin = X_test.reshape((X_test.shape[0], -1))
X_train_enc = np.hstack((X_train_origin, X_train_enc))
X_test_enc = np.hstack((X_test_origin, X_test_enc))

[ 2018-04-25 01:05:36,070][cascade_classifier.transform] X_groups_test.shape=[(1591, 5)]
[ 2018-04-25 01:05:36,072][cascade_classifier.transform] group_dims=[5]
[ 2018-04-25 01:05:36,074][cascade_classifier.transform] X_test.shape=(1591, 5)
[ 2018-04-25 01:05:36,076][cascade_classifier.transform] [layer=0] look_indexs=[0], X_cur_test.shape=(1591, 5)


In [11]:
# New data
X_train=X_train_enc
X_test=X_test_enc 
X_train_enc, X_test_enc = gc.fit_transform(X_train, y_train, X_test=X_test, y_test=y_test)

[ 2018-04-25 01:05:37,517][cascade_classifier.fit_transform] X_groups_train.shape=[(3228, 7)],y_train.shape=(3228,),X_groups_test.shape=[(1591, 7)],y_test.shape=(1591,)
[ 2018-04-25 01:05:37,519][cascade_classifier.fit_transform] group_dims=[7]
[ 2018-04-25 01:05:37,521][cascade_classifier.fit_transform] group_starts=[0]
[ 2018-04-25 01:05:37,522][cascade_classifier.fit_transform] group_ends=[7]
[ 2018-04-25 01:05:37,524][cascade_classifier.fit_transform] X_train.shape=(3228, 7),X_test.shape=(1591, 7)
[ 2018-04-25 01:05:37,526][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(3228, 7), X_cur_test.shape=(1591, 7)
[ 2018-04-25 01:05:38,269][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=98.45%
[ 2018-04-25 01:05:39,058][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=97.83%
[ 2018-04-25 01:05:39,858][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_fold

In [12]:
# predict  data + Es
y_pred = gc.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print("Test Accuracy of ExtraTrees = {:.6f} %".format(acc * 100))

[ 2018-04-25 01:05:45,507][cascade_classifier.transform] X_groups_test.shape=[(1591, 7)]
[ 2018-04-25 01:05:45,509][cascade_classifier.transform] group_dims=[7]
[ 2018-04-25 01:05:45,511][cascade_classifier.transform] X_test.shape=(1591, 7)
[ 2018-04-25 01:05:45,513][cascade_classifier.transform] [layer=0] look_indexs=[0], X_cur_test.shape=(1591, 7)


Test Accuracy of ExtraTrees = 98.868636 %


In [13]:
def update(X_train,X_test):
    # update Estimators
    M = np.delete(X_train, (-1), axis=1)
    X_train= np.delete(M, (-1), axis=1)
    M = np.delete(X_test, (-1), axis=1)
    X_test = np.delete(M, (-1), axis=1)
    return X_train,X_test

def newdata(X_test_enc, X_train_enc):
    # Make Data + Estimators
    X_train_origin = X_train.reshape((X_train.shape[0], -1))
    X_test_origin = X_test.reshape((X_test.shape[0], -1))
    X_train_enc = np.hstack((X_train_origin, X_train_enc))
    X_test_enc = np.hstack((X_test_origin, X_test_enc))
    return X_test_enc, X_train_enc

def affiche():
    # predict
    y_pred = gc.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    print("Test Accuracy of ExtraTrees = {:.6f} %".format(acc * 100),i,'fois')

In [14]:
# Deep
for i in range (20):
    X_train,X_test=update(X_train,X_test)
    X_test_enc, X_train_enc=newdata(X_test_enc, X_train_enc)

    X_train=X_train_enc
    X_test=X_test_enc 
    X_train_enc, X_test_enc = gc.fit_transform(X_train, y_train, X_test=X_test, y_test=y_test)
    affiche()

[ 2018-04-25 01:05:46,986][cascade_classifier.fit_transform] X_groups_train.shape=[(3228, 7)],y_train.shape=(3228,),X_groups_test.shape=[(1591, 7)],y_test.shape=(1591,)
[ 2018-04-25 01:05:46,988][cascade_classifier.fit_transform] group_dims=[7]
[ 2018-04-25 01:05:46,989][cascade_classifier.fit_transform] group_starts=[0]
[ 2018-04-25 01:05:46,991][cascade_classifier.fit_transform] group_ends=[7]
[ 2018-04-25 01:05:46,993][cascade_classifier.fit_transform] X_train.shape=(3228, 7),X_test.shape=(1591, 7)
[ 2018-04-25 01:05:46,994][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(3228, 7), X_cur_test.shape=(1591, 7)
[ 2018-04-25 01:05:47,594][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=98.76%
[ 2018-04-25 01:05:48,384][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=97.52%
[ 2018-04-25 01:05:49,249][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_fold

('Test Accuracy of ExtraTrees = 98.805783 %', 0, 'fois')


[ 2018-04-25 01:05:57,121][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=97.83%
[ 2018-04-25 01:05:57,894][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=97.52%
[ 2018-04-25 01:05:58,651][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_2.predict)=98.76%
[ 2018-04-25 01:05:59,510][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_3.predict)=97.52%
[ 2018-04-25 01:06:00,264][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_4.predict)=99.07%
[ 2018-04-25 01:06:01,022][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_5.predict)=96.90%
[ 2018-04-25 01:06:01,768][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_6.predict)=98.45%
[ 2018-04-25 01:06:02,587][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_7.predict)=98.14%


('Test Accuracy of ExtraTrees = 98.114393 %', 1, 'fois')


[ 2018-04-25 01:06:06,347][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=98.14%
[ 2018-04-25 01:06:07,150][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=98.45%
[ 2018-04-25 01:06:08,133][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_2.predict)=98.45%
[ 2018-04-25 01:06:08,975][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_3.predict)=98.14%
[ 2018-04-25 01:06:09,745][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_4.predict)=99.38%
[ 2018-04-25 01:06:10,575][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_5.predict)=97.52%
[ 2018-04-25 01:06:11,353][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_6.predict)=98.76%
[ 2018-04-25 01:06:12,115][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_7.predict)=98.76%


...

('Test Accuracy of ExtraTrees = 98.805783 %', 17, 'fois')


[ 2018-04-25 01:08:37,137][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=97.52%
[ 2018-04-25 01:08:37,902][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=96.90%
[ 2018-04-25 01:08:38,743][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_2.predict)=98.76%
[ 2018-04-25 01:08:39,521][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_3.predict)=97.52%
[ 2018-04-25 01:08:40,298][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_4.predict)=98.14%
[ 2018-04-25 01:08:41,078][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_5.predict)=97.52%
[ 2018-04-25 01:08:41,869][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_6.predict)=97.21%
[ 2018-04-25 01:08:42,660][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_7.predict)=97.83%


('Test Accuracy of ExtraTrees = 98.742929 %', 18, 'fois')


[ 2018-04-25 01:08:46,542][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=97.83%
[ 2018-04-25 01:08:47,352][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=97.21%
[ 2018-04-25 01:08:48,181][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_2.predict)=97.83%
[ 2018-04-25 01:08:48,919][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_3.predict)=98.14%
[ 2018-04-25 01:08:49,691][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_4.predict)=98.14%
[ 2018-04-25 01:08:50,532][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_5.predict)=97.21%
[ 2018-04-25 01:08:51,359][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_6.predict)=96.28%
[ 2018-04-25 01:08:52,156][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_7.predict)=98.14%


('Test Accuracy of ExtraTrees = 98.868636 %', 19, 'fois')


# ('Test Accuracy of ExtraTrees = 98.931490 %', 14, 'fois')